<span type="title">Spring AOP（基于AspectJ）</span> | <span type="update">2018-10-01</span> - Version <span type="version">1.1</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Spring AOP 面向切面编程的原理和操作。AOP 是一种开发理念，面向切面指的是将横切关注点和业务逻辑相分离这一思想。其中横切关注点指的是那些散布于应用中多出的功能，这些功能往往具有被动属性，不需要应用对象主动实现，比如日志记录、安全检查和过滤等等。散布在多处的属于同个被动功能很难去进行集中开发和管理。</p><p class="card-text">使用继承过于脆弱（单继承，难以应对变化），使用委托则繁复（切点需要传入上下文，委托需要提供一个服务执行方法，实现需要根据上下文判断是否需要在此处执行逻辑）。动态代理用于截获对象，添加功能，自然就是此问题的解决办法。同时就像 IOC 本质上是一个超大的抽象工厂，AOP其实是一个超级动态代理。不过，基于注解的AOP比JavaSE提供的动态代理更加方便和好用，并且配合IOC，使用更加方便。</p></span>

# AOP 的理念和缘起

假设现在我们有一个计算器的接口以及一个实现类：

```java
public interface Calculator {
    double add(double a, double b);
    double sub(double a, double b);
    double mul(double a, double b);
    double div(double a, double b);
}
public class CalculatorImpl implements Calculator {
    @Override
    public double add(double a, double b) {
        return a + b;
    }
    @Override
    public double sub(double a, double b) {
        return a - b;
    }
    @Override
    public double mul(double a, double b) {
        return a * b;
    }
    @Override
    public double div(double a, double b) {
        return a / b;
    }
    public static void main(String[] args) {
        Calculator calculator = new CalculatorImpl();
        System.out.println(calculator.add(1,2));
    }
}
```
为了在使用这个计算器类之前，对计算的数据进行日志记录和验证，在计算之后，对于返回值进行格式化，没有使用设计模式的思路是：对于每个实现类添加一对的横切关注点。自然，我们可以创建一个用于记录和验证的接口，然后再各个计算方法上放置这些代码，这是一种优化。此外，你可能想过，我们可以使用一个统一的服务接口类，委托给实现类，然后在这个接口中提供服务，这些服务的语句添加到各个计算方法上，这其实就是策略模式。

你可能还设想，通过 Spring 的 IOC 提供这个策略，并且自动进行注入，这样就没有问题了。这的确很好，是一种很理想的方案。但问题是，如果我们现在需要更多的服务，而之前的接口写死了两个子服务方法，那么我们就不得不在这些方法中通过参数判断当前的类是否为需要的类，然后用条件语句选择执行。这会导致越来越复杂的代码。

除去这种思路，我们其实可以在不更改任何POJO代码的前提下达到目的，通过代理。代理的缺点是，我们需要手动实现所有的服务接口，但是Java提供的动态代理依托于RTTI，简化了这一步骤:

```java
public class CalculatorLoggingProxy{
    private Calculator calculator;
    public CalculatorLoggingProxy(Calculator calculator) {
        this.calculator = calculator;
    }
    public Calculator getProxy() {
        return (Calculator) Proxy.newProxyInstance(
                calculator.getClass().getClassLoader(),
                new Class[]{Calculator.class},
                (proxy, method, args) -> {
                    return method.invoke(calculator,args);
                }
        );
    }
    public static void main(String[] args) {
        Calculator calculator = new CalculatorLoggingProxy(new CalculatorImpl()).getProxy();
        System.out.println(calculator.add(1.0,2.0));
    }
}
```

这个代理在对象被使用之前截获了对象，然后添加了代理服务，通过代理内部的委托，在代理中使用 method.invoke(Object,args) 方法实现了动态的调用。我们现在可以在 getProxy 方法中添加一些调用前和调用后的处理。

这里的问题是，这种方法依然不能在一个地方管理所有相同范畴的服务，解决方案很简单，通过 Spring 的 IOC 进行自动 bean 的代理包装，通过在一个类中提供被动服务，而将这些服务的方法自动插入代理类的相关语句中间，这样，我们就可以“面向切面”编程了。而建立切面类和代理类之间关系的最好方式，莫过于注解。Spring 提供了注解和XML配置两种方式，代理类会在创建的时候自动先创建切面类，然后将需要注入的切面的代码注入invoke方法前后，代理类内部的依赖会根据XM或者注解自动注入，然后代理会替代原来的类在 IOC 容器中充当 beans。

这看起来挺复杂，但是我们实际需要做的，仅仅是创建切面类的注解、指定需要插入的位点、启用自动代理即可。

AOP 是 OOP 的一个非常强大的补充，换句话说，AOP 降低了程序员编程的工程难度：通过在一个地方管理切面、自动进行代理和织入，我们可以无侵入的为现有代码添加功能。虽然对于计算机而言，或者反编译而言，AOP 是难寻踪迹的。

以下将分注解+Java配置文件自动代理、注解+XML配置文件自动代理、纯XML手动配置切面三种方式进行介绍。

# AOP 基本概念

## AOP 专用名词

AOP 的基本概念包括：

通知 Advice，通知指的是那些切面所提供的服务，比如日志和参数检查，这些服务称之为通知。通知有几种类型，分别是前置、后置、返回、异常和环绕，其分别对应在插入的业务逻辑之前、之后、返回值的时候、出错的时候。环绕通知包裹了业务逻辑，可以在其前、返回值、出错、后各处进行自定义行为。

连接点 Joint point：被插入的业务逻辑位置

切点 pointcut：匹配通知的一个或者多个连接点

切面 aspect: 通知 和 切点 的组合。切面是一个 Java 类，其提供一组方法，用来在指定切点执行各种通知。

## AspectJ 表达式语言

Spring 经典的 AOP 是基于 Java 动态代理的，其采用 Java 类的方式实现了切面编程。而 AspectJ 包的 AOP 是基于 Java 语言扩展的，其功能和粒度更加细致。因此之后介绍的 AOP 是基于 AspectJ 的。

Spring 支持的 AspectJ 是局部的，但是足够使用。ApsectJ 定义了一套切点表达式语言，Spring 用的最多的是：`execution(), arg(), within(), bean()`。其中，`execution(public int package.class.method(arg1,arg2...)) || arg(number) && within(package.*) && !bean('bean_id')` 这个表达式大概能说明问题。

表达式使用双引号，放在注解中：`@PointCut("execution(public ...)")` 这样。

可以使用 Java 的异或非运算符，注意，在XML中使用 and，or，not。

其中 execution 标明符合此条件的方法，可以过滤描述词、返回值、类、方法、参数个数、参数类型。

其中 args 表明此切点允许传入参数，number 表示，任何使用此注解的方法，应该提供此引用名称的传入参数。

其中 within 表明其对于切点范围有限制，当包下的所有类的方法被调用，均会触发切面。

其中 bean 限制了此切面只对于指定名称的 bean 有效，注意，此处需要使用单引号！！！！

args 使用举例如下：

```java
@Pointcut(value = "execution(* *.*(..)) || args(number,info)")
public void count(int number, String info) {}

@Before("count(number,info)")
public void countA(int number, String info) { ... }
```

注意，args 适用于被代理的方法提供了和切点个数和类型匹配的参数情况，其中类型和名称通过 PointCut 注解的方法的传入参数提供，在过滤时需要将引用名称放置在 args 的括号中，在使用时，需要在注解上提供带有参数名称的切点、在通知传入括号中提供传入参数名称和类型以进行调用，当然，也可以不传递参数到通知方法。但是，只要在切点定义args，那么就必将过滤方法的参数类型和数量，对这部分过滤后的方法进行切面通知。

你可以假象通过 count 方法 -> pointcut -> before -> countA 方法进行的调用，除了头尾需要提供类型和引用，中间只需要提供引用即可。

# 使用注解 + Java 自动代理

## 使用概览

这是最为推荐的一种方式。首先需要将实现类声明为组件：

```java
@Component(value = "calculator")
public class CalculatorImpl implements Calculator { ... }
```

接着新建一个 Java 的配置文件，这个文件启用自动bean注解扫描，我们顺便提供了一个测试用的方法。

```java
@Configuration
@ComponentScan(basePackages = "com.mazhangjing.aop")
@EnableAspectJAutoProxy
public class Config {
    public static void main(String[] args) {
        ApplicationContext ctx = new AnnotationConfigApplicationContext(com.mazhangjing.aop.Config.Config.class);
        Calculator calculator = (Calculator)ctx.getBean("calculator");
        System.out.println(calculator.add(2,3));
    }
}
```

注意，到这里和传统的 IOC 没什么区别，仅仅是自动装配了一个 beans。注意，我们使用 `@EnableAspectJAutoProxy` 启用了代理自动创建，启用之后，所有需要被代理的类都会自动进行代理。

下面新建一个切面，提供被代理者类、代理方法、代理位置信息：

```java
@Component
@Aspect
public class  LoggingAspect {

    @Order(0)
    @Pointcut(value = "execution(public double com.mazhangjing.aop.Calculator.*(double,double))")
    public void calculator() { }

    @Before("calculator()")
    public void before(JoinPoint point) {
        System.out.println("Method " + point.getSignature().getName() + " calling with args: " +
                Arrays.asList(point.getArgs()));
    }

    @After("calculator()")
    public void after(JoinPoint point) {
        System.out.println("Method " + point.getSignature().getName() + " calling end.");
    }

    @AfterReturning(value = "calculator()", returning = "result")
    public void end(JoinPoint point,Object result) {
        System.out.println("Method " + point.getSignature().getName() + " calling end. result is " + result);
    }

    @AfterThrowing(value = "calculator()" ,throwing = "e")
    public void error(JoinPoint point, Exception e) {
        System.out.println("Error happens" + e.toString());
    }

    @Around("calculator()")
    public Object around(ProceedingJoinPoint joinPoint) {
        Object result = null;
        try {
            //Before
            result = joinPoint.proceed();
            //AfterReturning
        } catch (Throwable e) {
           e.printStackTrace();
           //AfterThrowing
        }
        //After
        return result;
    }
}
```

注意到，这个切面类首先被配置为 bean，然后使用 @Aspect 作为了切面。也就是说，AOP 是后置的，需要首先将原始的被代理者初始化为 bean后，将切面类也初始化为 bean后，才开始进行的代理：当我们有这两个 bean 后，AOP通过寻找 @Aspect 注解，根据连接点信息，将切面的服务分配的正在创建的代理对象中去，然后将原始的被代理 bean替换为代理后的代理对象。这样的话，通过在 IOC 寻找原来创建的 bean id，即可获取代理后的对象。

注意到这一切是如何的无侵入，并且简单。被代理者仅仅是一个 POJO，没有添加任何 AspectJ 的代码，当我们启用自动代理后，从 IOC 调用这个 POJO，竟然神奇的返回了代理后的对象。我们唯一需要做的就是在一个切面类中定义连接点以及需要插入的服务，然后将其声明为 @Aspect 即可。

**总之，AOP的自动扫描使用方法操作如下：对于切面使用 @Component @Aspect 注解，对于 @Configuration 使用 @ComponentScan 和 @EnableAspectJAutoProxy 注解。在切面类中对于需要代理的方法前添加 @Before 和 @After 以及 @AfterReturning 注解。在方法中可以传入 JointPoint 参数获取切点的方法签名、参数等信息。可以使用 @Order 指定切面优先级。可以使用 @PointCut 指定切面切点，简化注解。在切点指定过程中，可以使用多个通配符来过滤限定、类、方法、参数信息，以代理特定的POJO对象。**


## 确定注解插入点的位置

- Before 前置通知，在连接点之前执行。

- AfterReturning 返回通知，可以获得返回值，后置通知不能获取返回值。

- After 后置通知，在连接点之后执行。不论调用的方法是否存在异常、返回什么，不能获取目标方法执行结果。

- AfterThrowing 异常代理，当出现异常的时候进行通知。

- Around 环绕通知，即重现Java动态代理的整个步骤，可以手动实现前置、返回、后置和异常通知。环绕通知最接近动态搭理，如果你希望一次处理多个位点信息，使用环绕通知。


需要注意，对于 AfterReturning，需要在注解提供返回值签名和切点两个参数。对于 Around 需要传入特殊的 JointPoint 参数，对于 AfterThrowing，需要提供异常值和切点两个参数。

各个位置对应动态代理如下：

```java
(proxy, method, args) -> {
    //前置通知
    Object o = 0;
    try {
        o =  method.invoke(calculator,args);
        //返回通知
    } catch (Exception e) {
        //异常通知
    }
    //后置通知
    return o;
}
```


## 使用注解表达式匹配方法

声明限定词、返回值、包名、类名、方法名、传入参数类型。

其中限定词和返回值可用一个星号代替，也可以只写一个，另一个用星号代替。包名和类名可用星号代替，方法名可用星号代替，参数长度可用..代替，此外，参数可指定数个类型，之后用..代替。

- 比如：`public double com.a.calculator.add(int,int)`
- 比如：`* double com.a.calculator.add(int,int)`
- 比如：`public * com.a.calculator.add(int,int)`
- 比如：`* com.a.calculator.add(int,int)`
- 比如：`public double *.add(int,int)`
- 比如：`public double com.a.calculator.*(int,int)`
- 比如：`public double com.a.calculator.add(..)`
- 比如：`public double com.a.calculator.add(double,..)`
- 比如：`* *.*(..)`

表达式同样支持 `|| && 以及 !` 添加逻辑判断：`@Before(value="execution(* *.a(..)) || execution(* *.b(..))")`


## 使用切入点表达式简化代码

在 @Before 和 @After 后加入 execution 过于复杂，可以在本包下新建一个空方法，然后使用 @PointCut 注解，在此注解中写入 execution 即可，在@After 等切面注解上，使用此方法的签名即可，如果跨包，则需要添加包名。


## 添加切面服务优先级

对于单个切面，有固定顺序。对于不同切面，使用Order注解指定：`@Order(0)` 数字越小越优先。

# 使用注解 + XML 自动代理

和上面类似，区别在于，XML 作为 Spring 的 IOC 最终位点，自动扫描和自动代理需要在 XML 设置。

```xml
xmlns:context="http://www.springframework.org/schema/context"
xmlns:aop="http://www.springframework.org/schema/aop"
<context:component-scan base-package="com.mazhangjing.aop" />
<aop:aspectj-autoproxy />
```

```java
ApplicationContext context = new ClassPathXmlApplicationContext("aopContext.xml");
Calculator calculator = (Calculator) context.getBean("calculator");
System.out.println(calculator.add(1,2));
```

# 使用 XML 手动创建切面代理

这种方法的配置和之前类似，不过，我们现在并不是让 Spring IOC 自动装载 Bean，也不是让 AspectJ 自动创建代理，我们需要手动指定切面类以及切点和服务，还有手动创建原始POJO、切面类的 beans。

这种情况下，不需要使用任何注解，对于Java代码侵入性最小。（通过RTTI实现）

```xml
<bean id="calculator" class="com.mazhangjing.aop.CalculatorImpl" />
<bean id="aspect" class="com.mazhangjing.aop.Aspect.LoggingAspect" />
<aop:config>
    <aop:pointcut id="pointcut" expression="execution(public double com.mazhangjing.aop.Calculator.*(double,double))"/>
    <aop:aspect ref="aspect" order="0">
        <aop:before method="before" pointcut-ref="pointcut" />
        <aop:after method="after" pointcut-ref="pointcut" />
        <aop:after-returning method="end" returning="result" pointcut-ref="pointcut" />
        <aop:after-throwing method="error" throwing="e" pointcut-ref="pointcut" />
        <aop:around method="around" pointcut-ref="pointcut" />
    </aop:aspect>
</aop:config>
```

注意，我们依然要创建 POJO 和切面的 bean，然后在 aop:config 中新建 pointcut，匹配POJO的方法，定义连接点。同时在 aop:aspect 中配置切面类的切入方法和其切入的位点，可选 aop:before, aop:after, aop:after-retruning, aop-after-throwing, aop-around。每个切面要使用一个 aop-aspect 标签，放置在 aop:config 中。